In [1]:
import pandas as pd
import sqldf
from tqdm import tqdm, trange

In [72]:
df=pd.read_excel('Buyers transparency new coding0822.xlsx',sheet_name='data')
df.iloc[:, -10:] = df.iloc[:, -10:].fillna(-1)

In [74]:
def finalize():
    global df
    df['buyertracemx']=df[['buyer1trace','buyer2trace','buyer3trace']].max()
    df['buyerknowmx']=df[['buyer1know','buyer2know','buyer3know']].max()

In [81]:
with open('info.txt') as info:
    info=[x for x in info.read().split('#') if x!='']

info={x.split('\n')[0].strip():{y.strip().rsplit(' ',maxsplit=1)[0].strip().lower():int(y.strip().rsplit(' ',maxsplit=1)[1]) for y in x.split('\n')[1:] if len(y.rsplit(' ',maxsplit=1))>1} for x in info}
# info
print([len(info[x]) for x in info])

[150, 150, 150, 150]


# direct match

In [83]:
df['buyer1pol']=df['buyer1name'].map(info['pol'])
df['buyer1gov']=df['buyer1name'].map(info['gov'])
for i in '123':
    for j in ['trace','know']:
        df['buyer'+i+j]=df['buyer'+i+'name'].map(info[j])

array([nan,  0., 35., 11.,  6., 13., 46., 48., 32.,  1., 19., 12., 47.,
       20., 24., 27., 39., 49., 22., 21., 34.])

# fuzz match

# manual check

# finalize + export

In [ ]:
finalize()
df.to_csv('indexCode.csv',index=0)